In [8]:
from faker import Faker
import random
import uuid
import pandas as pd

In [9]:
# Initialisiere Faker mit deutschem Locale für realistische Namen
fake = Faker("de_DE")

In [10]:
# Kompetenzgruppen nach Fachrichtung
#kompetenzen_pool = ["Schaltungstechnik", "Leistungselektronik", "Signalverarbeitung", "CAD", "FEM", "Thermodynamik", "Mechanik", "Machine Learning", "Data Engineering", "Statistik", "Softwarearchitektur", "NLP", "Python", "Datenbanken", "KI", "Cloud Computing", "Agile Methoden", "Projektmanagement", "Teamführung", "Kommunikation", "Präsentationstechniken", "Quantenphysik", "Optik", "Statistische Mechanik", "Numerische Mathematik", "Mathematische Physik", "Astronomie", "Geophysik", "Materialwissenschaften", "Nanotechnologie", "Biophysik", "Medizinische Physik", "Photonik", "Akustik", "Elektromagnetismus", "Lineare Algebra", "Analysis", "Differentialgleichungen", "Stochastik", "Numerische Analysis", "Optimierung", "Graphentheorie", "Kombinatorik", "Mathematische Logik", "Topologie", "Funktionalanalysis", "Komplexe Analysis", "Differentialgeometrie"]
kompetenzen_pool = {
    "Maschinenbau": ["CAD", "FEM", "Thermodynamik", "Mechanik"],
    "Elektrotechnik": ["Schaltungstechnik", "Leistungselektronik", "Signalverarbeitung"],
    "Informatik": ["Python", "Datenbanken", "KI", "Softwarearchitektur"],
    "Physik": ["Quantenphysik", "Optik", "Statistische Mechanik"],
    "Mathematik": ["Lineare Algebra", "Stochastik", "Numerik"]
}

In [13]:
# Flache Liste aller verfügbaren Kompetenzen
alle_kompetenzen = [k for gruppe in kompetenzen_pool.values() for k in gruppe]

# Verfügbare Projekte (für Zuweisung/Simulation)
projekt_ids = ["PROJ-A", "PROJ-B", "PROJ-C"]

In [16]:
# Erzeuge 150 synthetische Personen mit Fähigkeiten und Verfügbarkeiten
def generiere_personen(anzahl: int = 150) -> pd.DataFrame:
    """
    Generiert eine Liste von Personen mit zufälligen Kompetenzen und monatlicher Verfügbarkeit.

    Args:
        anzahl (int): Anzahl der zu generierenden Personen.

    Returns:
        pd.DataFrame: DataFrame mit Personeninformationen.
    """
    personen = []
    for _ in range(anzahl):
        person = {
            "id": str(uuid.uuid4()),
            "name": fake.name(),
            "kompetenzen": random.sample(alle_kompetenzen, random.randint(1, 4)),
            "zeitbudget": round(random.uniform(0.4, 1.0), 2)  # Arbeitszeitanteil zwischen 40 % und 100 %
        }
        # Verfügbarkeit für jeden Monat im Jahr 2025 generieren
        for monat in range(1, 13):
            monat_label = f"{monat:02d}/2025"
            
            # Zuweisung zu einem Projekt oder "Frei" (also verfügbar für neues Matching)
            projekt = random.choices(projekt_ids + ["Frei"], weights=[0.25, 0.25, 0.25, 0.25])[0]
            
            person[f"projektbelegung_{monat_label}"] = projekt
            
            # Verfügbarkeit nur eintragen, wenn "Frei" oder später passender Projekt-Match
            if projekt == "Frei":
                person[f"verfuegbarkeit_{monat_label}"] = round(random.uniform(0.4, 1.0), 2)
            else:
                person[f"verfuegbarkeit_{monat_label}"] = 0.0  # Keine Kapazität in gebuchten Monaten

        personen.append(person)

    df = pd.DataFrame(personen)
    # Kompetenzen als durch Kommas getrennten String speichern
    df["kompetenzen"] = df["kompetenzen"].apply(lambda x: ", ".join(x))
    return df

# Speichere generierte Personen in CSV
df_personen = generiere_personen()
df_personen.to_csv("personen.csv", index=False)
print("150 Personen gespeichert als 'personen.csv'")

150 Personen gespeichert als 'personen.csv'


In [17]:
# Beispielhafte Projektaufgaben mit zugehöriger Kompetenz und Aufwand
teilaufgaben = [
    {"bezeichnung": "Entwurf CAD-Baugruppe", "kompetenz": "CAD", "aufwand": 3.0},
    {"bezeichnung": "Simulation FEM-Struktur", "kompetenz": "FEM", "aufwand": 2.5},
    {"bezeichnung": "Signalverarbeitung Sensor", "kompetenz": "Signalverarbeitung", "aufwand": 2.0},
    {"bezeichnung": "Entwicklung Python-Modul", "kompetenz": "Python", "aufwand": 3.5},
    {"bezeichnung": "Statistische Auswertung", "kompetenz": "Stochastik", "aufwand": 2.0},
    {"bezeichnung": "Optische Analyse", "kompetenz": "Optik", "aufwand": 1.5},
    {"bezeichnung": "Datenbankmodellierung", "kompetenz": "Datenbanken", "aufwand": 2.0},
    {"bezeichnung": "Thermodynamische Bewertung", "kompetenz": "Thermodynamik", "aufwand": 3.0}
]

In [18]:
# Ergänze Aufgaben um Projekt- und Teilaufgaben-IDs sowie Zeiträume
def erweitere_teilaufgaben(teilaufgaben: list, projekt_id: str) -> pd.DataFrame:
    """
    Ergänzt Teilaufgaben um UUIDs und Zeitinformationen.

    Args:
        teilaufgaben (list): Liste von Aufgaben als Dictionaries.
        projekt_id (str): Gemeinsame Projekt-ID für alle Aufgaben.

    Returns:
        pd.DataFrame: DataFrame mit erweiterten Teilaufgaben.
    """
    for ta in teilaufgaben:
        ta["teilaufgabe_id"] = str(uuid.uuid4())
        ta["projekt_id"] = projekt_id
        ta["start"] = "03/2025"
        ta["ende"] = "06/2025"
    return pd.DataFrame(teilaufgaben)

projekt_id = str(uuid.uuid4())
df_teilaufgaben = erweitere_teilaufgaben(teilaufgaben, projekt_id)
df_teilaufgaben.to_csv("teilaufgaben.csv", index=False)
print("Projekt gespeichert als 'teilaufgaben.csv'")

Projekt gespeichert als 'teilaufgaben.csv'
